In [2]:
import pandas as pd;

pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : 0691c5cf90477d3503834d983f69350f250a6ff7
python                : 3.13.5
python-bits           : 64
OS                    : Darwin
OS-release            : 25.1.0
Version               : Darwin Kernel Version 25.1.0: Mon Oct 20 19:26:04 PDT 2025; root:xnu-12377.41.6~2/RELEASE_ARM64_T8122
machine               : arm64
processor             : arm
byteorder             : little
LC_ALL                : None
LANG                  : C.UTF-8
LOCALE                : C.UTF-8

pandas                : 2.2.3
numpy                 : 2.1.3
pytz                  : 2024.1
dateutil              : 2.9.0.post0
pip                   : 25.1
Cython                : None
sphinx                : 8.2.3
IPython               : 8.30.0
adbc-driver-postgresql: None
adbc-driver-sqlite    : None
bs4                   : 4.12.3
blosc                 : None
bottleneck            : 1.4.2
dataframe-api-compat  : None
fastparquet           : None
fsspec         

since we already have zstandard installed, we can use pandas directly to read the zst files.

In [3]:
# Create a single dataframe with the concatenation of all input csv files, adding a column called country
countries = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']
dfs = []

for country in countries:
    df = pd.read_csv(f'trendingYT/{country}videos.csv.zst', compression='zstd', encoding='utf-8', encoding_errors='ignore')
    df['country'] = country
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

In [4]:
df_all.shape

(375942, 17)

In [5]:
# Extract all videos that have no tag.
df_no_tags = df_all[df_all['tags'].isna() | (df_all['tags'] == '[none]') | (df_all['tags'] == '')]
print(f"Videos with no tags: {len(df_no_tags)}")

Videos with no tags: 37698


In [6]:
# For each channel, determine the total number of views
channel_views = df_all.groupby('channel_title')['views'].sum().sort_values(ascending=False)
channel_views

channel_title
ChildishGambinoVEVO     11016766510
Marvel Entertainment    10430605449
NickyJamTV               9479859505
Ozuna                    8623329509
ibighit                  8205572221
                           ...     
NavylittleMonster               365
Videostendencias                302
No Comment TV                   284
Sport Life                      163
Alexander Redking               153
Name: views, Length: 37824, dtype: int64

In [7]:
# Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.
excluded = df_all[
    (df_all['comments_disabled'] == True) & (df_all['ratings_disabled'] == True) |
    (df_all['video_error_or_removed'] == True)
]

print (excluded.shape)

df_all = df_all[~df_all.index.isin(excluded.index)]
df_all.shape

(2620, 17)


(373322, 17)

In [8]:
# Add a like_ratio column storing the ratio between the number of likes and of dislikes
df_all.loc[:,'like_ratio'] = df_all['likes'] / df_all['dislikes'].replace(0, pd.NA)
df_all

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA,18.135076
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,75.707346
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA,27.3525
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,66.485168
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA,77.513044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18T13:00:04.000Z,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals,US,27.552347
375938,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18T01:00:06.000Z,[none],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN,US,157.089005
375939,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18T17:34:22.000Z,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...,US,46.577519
375940,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17T17:00:04.000Z,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...,US,67.799368


In [9]:
# Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)
df_all['publish_time'] = pd.to_datetime(df_all['publish_time'])

floor_times = df_all['publish_time'].dt.floor('10min')
ceil_times = floor_times + pd.Timedelta(minutes=10)

df_all['interval'] = floor_times.dt.strftime('%H:%M') + ' to ' + ceil_times.dt.strftime('%H:%M')


In [10]:
df_all

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio,interval
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA,18.135076,17:00 to 17:10
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,75.707346,17:00 to 17:10
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA,27.3525,19:00 to 19:10
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,66.485168,18:00 to 18:10
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA,77.513044,11:00 to 11:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18 13:00:04+00:00,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals,US,27.552347,13:00 to 13:10
375938,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18 01:00:06+00:00,[none],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN,US,157.089005,01:00 to 01:10
375939,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18 17:34:22+00:00,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...,US,46.577519,17:30 to 17:40
375940,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17 17:00:04+00:00,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...,US,67.799368,17:00 to 17:10


In [11]:
# For each interval, determine the number of videos, average number of likes and of dislikes.

df_all.groupby('interval').agg({
    'video_id': 'count',
    'likes': 'mean',
    'dislikes': 'mean'
}).rename(columns={'video_id': 'num videos', 'likes': 'avg likes', 'dislikes': 'avg dislikes'})

,num videos,avg likes,avg dislikes
interval,,,
00:00 to 00:10,2897,61288.115637,3808.149465
00:10 to 00:20,1509,22748.138502,1449.836315
00:20 to 00:30,1241,21378.280419,1072.344883
00:30 to 00:40,1614,36853.560719,955.890954
00:40 to 00:50,1269,42198.623325,1909.301812
...,...,...,...
23:10 to 23:20,1554,22783.840412,1110.168597
23:20 to 23:30,1438,30696.510431,1177.020862
23:30 to 23:40,1666,18159.173469,874.184874


In [ ]:
#  For each tag, determine the number of videos

# tags_dict = {}
# for tags_str in df_all['tags'].dropna():
#     if tags_str and tags_str != '[none]':
#         # Split by pipe character
#         tags = [tag.strip().strip('"') for tag in str(tags_str).split('|')]
#         for tag in tags:
#             if tag in tags_dict:
#                 tags_dict[tag] += 1
#             else:
#                 tags_dict[tag] = 1


# the above solution is fine but it is slow so here is a more pandas way of doing it

tags_list = []
for tags_str in df_all['tags'].dropna():
    # noticed that there are some empty strings in the tag column, so we remove them
    if tags_str and tags_str != '[none]':
        # Split by pipe character
        tags = [tag.strip().strip('"') for tag in str(tags_str).split('|')]
        tags_list.extend(tags)

tag_counts = pd.Series(tags_list).value_counts()
tag_counts

funny                    15039
comedy                   12351
2018                     11383
news                      6363
music                     5909
                         ...  
संकल्प से सिद्धि             1
Sankalp Se Siddhi            1
ICICI Fraud                  1
आईसीआईसीआई फर्जीवाड़ा        1
langford                     1
Name: count, Length: 848605, dtype: int64

In [13]:
# Find the tags with the largest number of videos
tag_counts.sort_values(ascending=False).head(1)

funny    15039
Name: count, dtype: int64

In [26]:
# For each (tag, country) pair, compute average ratio likes/dislikes
tag_df = df_all[((df_all['tags'] != '[none]') & (df_all['tags'] != '') & (df_all['tags'].isna() == False))].copy()
tag_df['tag'] = tag_df['tags'].str.replace('"', '').str.split('|')
tag_df = tag_df.explode('tag')
# noticed that there are some empty strings in the tag column, so we remove them
tag_df = tag_df[tag_df['tag'] != '']
tag_df.groupby(['tag', 'country'], as_index = False).agg({'like_ratio':'mean'}).sort_values('tag')

,tag,country,like_ratio
0,#Freeticket,IN,2.665679
1,#GST,IN,2.416896
2,#Jaisimha,IN,2.665679
3,#JanaSenaParty,IN,17.621907
4,#MahaaNews,IN,8.511947
...,...,...,...
1098635,😥,DE,26.384615
1098636,😭 #résistancelamaindesqueezie ❤️,FR,10.219048
1098637,😭قفز عسكري الوحيد من الطائرة العسكرية الجزائرية,FR,6.223873
1098638,🚀 smartpixel.com.ua,RU,0.714286


In [ ]:
# For each (trending_date, country) pair, the video with the largest number of views

df_all.loc[df.groupby(['trending_date', 'country'])['views'].idxmax()]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio,interval
3400,xpck4IdClZg,17.01.12,I'm shook,TheFishPenetrator,22,2017-11-30 00:34:43+00:00,"computer|""wallpaper""|""monitor""|""kid""|""scared""|...",833425,15211,491,688,https://i.ytimg.com/vi/xpck4IdClZg/default.jpg,False,False,False,"My friend gave me his wallpaper dump, I though...",CA,30.979633,00:30 to 00:40
3605,4WnizmrnJlk,17.02.12,Matt Lauer Sexual Harassment Allegations; Trum...,Late Night with Seth Meyers,23,2017-12-01 02:00:01+00:00,"Late night|""Seth Meyers""|""closer Look""|""Matt l...",1855536,21004,1295,2692,https://i.ytimg.com/vi/4WnizmrnJlk/default.jpg,False,False,False,Seth takes a closer look at how the national r...,CA,16.219305,02:00 to 02:10
3818,Ze6OXuZt6DI,17.03.12,THE LAST WORD WITH LAWRENCE O’DONNELL [FULL] 1...,Elijah Price,22,2017-12-02 04:08:32+00:00,"Breaking News|""Trump News""|""Breaking News Trum...",131821,933,111,523,https://i.ytimg.com/vi/Ze6OXuZt6DI/default.jpg,False,False,False,THE LAST WORD WITH LAWRENCE O’DONNELL [FULL] 1...,CA,8.405405,04:00 to 04:10
4043,JbFNkF5mscM,17.04.12,Suit (Full Video) |Anmol Gagan Maan|Teji Sandh...,Speed Records,10,2017-12-03 04:30:00+00:00,"punjabi songs|""punjabi bhangra""|""punjabi music...",759954,21292,2570,1453,https://i.ytimg.com/vi/JbFNkF5mscM/default.jpg,False,False,False,Song - Suit\nArtist - Anmol Gagan Maan\nLyrics...,CA,8.284825,04:30 to 04:40
4276,INotkJxL_Bs,17.05.12,'How It's Gotta Be' Next on Ep. 808 | The Walk...,amc,24,2017-12-04 03:00:01+00:00,"The Walking Dead|""12/3_TWD""|""episode 808""|""epi...",65965,951,22,509,https://i.ytimg.com/vi/INotkJxL_Bs/default.jpg,False,False,False,Don’t miss the Mid-Season Finale of The Walkin...,CA,43.227273,03:00 to 03:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31877,RoflWNvJg_w,18.30.04,"Por primera vez, Daddy Yankee, Natti Natasha y...",Telemundo Entretenimiento,24,2018-04-27 18:49:21+00:00,"Premios Billboard|""msn""|""premios billboard de ...",3906321,75225,4305,3162,https://i.ytimg.com/vi/RoflWNvJg_w/default.jpg,False,False,False,Video ofician de Telemundo Premios Billboard. ...,CA,17.473868,18:40 to 18:50
37935,sV7eDd1d7LI,18.31.05,Cooking birthday presents for my wife!,JunsKitchen,26,2018-05-28 10:00:34+00:00,"cook|""cat""|""Jun""|""Rachel""|""Japanese""|""food""|""j...",1819351,166934,1033,12291,https://i.ytimg.com/vi/sV7eDd1d7LI/default.jpg,False,False,False,I opened a second channel for smaller videos o...,CA,161.601162,10:00 to 10:10
15339,8KuDjfxCJhk,18.31.01,JUICY KING SALMON BELLY and Japanese Hawaii Fo...,Mark Wiens,19,2018-01-28 13:00:00+00:00,"Maguro Brothers Hawaii|""hawaiian food""|""japane...",313184,10685,214,759,https://i.ytimg.com/vi/8KuDjfxCJhk/default.jpg,False,False,False,Maguro Brothers is an amazing little Japanese ...,CA,49.929907,13:00 to 13:10
26991,m3F07R2-zW4,18.31.03,Why the Stormy Daniels lawsuit matters,Vox,25,2018-03-30 12:00:09+00:00,"stormy daniels|""trump""|""impeachment""|""lawsuits...",230204,6474,1126,1095,https://i.ytimg.com/vi/m3F07R2-zW4/default.jpg,False,False,False,Trump’s presidency may be in jeopardy even if ...,CA,5.749556,12:00 to 12:10


In [35]:
# Divide trending_date into three columns: year, month, day

df_all['trending_date_parsed'] = pd.to_datetime(df_all['trending_date'], format='%y.%d.%m')
df_all['year'] = df_all['trending_date_parsed'].dt.year
df_all['month'] = df_all['trending_date_parsed'].dt.month
df_all['day'] = df_all['trending_date_parsed'].dt.day

In [36]:
df_all

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,ratings_disabled,video_error_or_removed,description,country,like_ratio,interval,trending_date_parsed,year,month,day
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,...,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA,18.135076,17:00 to 17:10,2017-11-14,2017,11,14
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,...,False,False,STill got a lot of packages. Probably will las...,CA,75.707346,17:00 to 17:10,2017-11-14,2017,11,14
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,...,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA,27.3525,19:00 to 19:10,2017-11-14,2017,11,14
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,...,False,False,I know it's been a while since we did this sho...,CA,66.485168,18:00 to 18:10,2017-11-14,2017,11,14
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,...,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA,77.513044,11:00 to 11:10,2017-11-14,2017,11,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18 13:00:04+00:00,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,...,False,False,The Cat Who Caught the Laser - Aaron's Animals,US,27.552347,13:00 to 13:10,2018-06-14,2018,6,14
375938,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18 01:00:06+00:00,[none],1064798,60008,382,...,False,False,NaN,US,157.089005,01:00 to 01:10,2018-06-14,2018,6,14
375939,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18 17:34:22+00:00,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,...,False,False,I had so much fun transforming Safiyas hair in...,US,46.577519,17:30 to 17:40,2018-06-14,2018,6,14
375940,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17 17:00:04+00:00,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,...,False,False,How Black Panther Should Have EndedWatch More ...,US,67.799368,17:00 to 17:10,2018-06-14,2018,6,14


In [37]:
# For each (month, country) pair, the video with the largest number of views
df_all.loc[df_all.groupby(['month', 'country'])['views'].idxmax()]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,ratings_disabled,video_error_or_removed,description,country,like_ratio,interval,trending_date_parsed,year,month,day
11214,LsoLEjrDogU,18.09.01,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",43067983,1717177,61567,...,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,CA,27.891192,04:40 to 04:50,2018-01-09,2018,1,9
51930,LsoLEjrDogU,18.08.01,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",37728802,1629946,56305,...,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,DE,28.948513,04:40 to 04:50,2018-01-08,2018,1,8
92853,LsoLEjrDogU,18.08.01,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars""|""Finesse""|""Cardi B""|""Finesse Remix...",37728802,1629948,56305,...,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,FR,28.948548,04:40 to 04:50,2018-01-08,2018,1,8
135208,LsoLEjrDogU,18.18.01,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",90598955,2248693,93089,...,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,GB,24.156377,04:40 to 04:50,2018-01-18,2018,1,18
173405,dfnCAmr569k,18.18.01,"Taylor Swift - End Game ft. Ed Sheeran, Future",TaylorSwiftVEVO,10,2018-01-12 05:00:01+00:00,"Taylor|""Swift""|""End""|""Game""|""Big""|""Machine""|""Pop""",42019590,1804377,100033,...,False,False,Music video by Taylor Swift performing End Gam...,IN,18.037818,05:00 to 05:10,2018-01-18,2018,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166769,FlsCjmMhFmw,17.12.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06 17:58:51+00:00,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",125432237,2912710,1545017,...,False,False,"YouTube Rewind 2017. Celebrating the videos, p...",IN,1.885228,17:50 to 18:00,2017-12-12,2017,12,12
224092,FlsCjmMhFmw,17.11.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06 17:58:51+00:00,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",113876217,2811223,1470388,...,False,False,"YouTube Rewind 2017. Celebrating the videos, p...",KR,1.911892,17:50 to 18:00,2017-12-11,2017,12,11
259064,FlsCjmMhFmw,17.10.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06 17:58:51+00:00,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",100912384,2656698,1353667,...,False,False,"YouTube Rewind 2017. Celebrating the videos, p...",MX,1.962593,17:50 to 18:00,2017-12-10,2017,12,10
299087,FlsCjmMhFmw,17.08.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06 17:58:51+00:00,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",52611730,1891836,884967,...,False,False,"YouTube Rewind 2017. Celebrating the videos, p...",RU,2.137748,17:50 to 18:00,2017-12-08,2017,12,8
